# Homework 4, BEE 6940 (Due By 5/4/23, 9:00PM)

**Name**:

**ID**:

## Overview

### Instructions

- Problem 1 asks you to conduct several model checks (graphical and quantitative) on a stationary GEV model for tide gauge data.
- Problem 2 asks you to do the same on a non-stationary GEV model.
- Problem 3 asks you to compare the results of Problems 1 and 2 to draw conclusions about the evidence for non-stationarity.

### Load Environment

The following code loads the environment and makes sure all needed packages are installed. This should be at the start of most Julia scripts.

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using Random
using StatsBase
using CSV # load CSV data
using Dates
using DataFrames # data storage and presentation
using DataFramesMeta
using Plots # plotting library
using StatsPlots # statistical plotting
using Distributions # statistical distribution interface
using Turing # probabilistic programming and MCMC
using Optim # optimization library

### Load Data

Next, let's load and process the data (as we did in Lab 09). This notebook uses hourly data from the San Francisco, CA tide gauge station, obtained from the [University of Hawaii Sea Level Center](https://uhslc.soest.hawaii.edu/) [(Caldwell et al (2015))](https://doi.org/10.7289/V5V40S7W).

In [ ]:
## load the data from the file and return a DataFrame of DateTime values and gauge measurements

function load_data(fname)
    date_format = DateFormat("yyyy-mm-dd HH:MM:SS")
    # This uses the DataFramesMeta.jl package, which makes it easy to string together commands to load and process data
    df = @chain fname begin
        CSV.read(DataFrame; header=false)
        rename("Column1" => "year", "Column2" => "month", "Column3" => "day", "Column4" => "hour", "Column5" => "gauge")
        # need to reformat the decimal date in the data file
        @transform :datetime = DateTime.(:year, :month, :day, :hour)
        # replace -99999 with missing
        @transform :gauge = ifelse.(abs.(:gauge) .>= 9999, missing, :gauge)
        select(:datetime, :gauge)
    end
    return df
end

dat = load_data("data/h551.csv")

In [ ]:
## detrend the data to remove the effects of sea-level rise and seasonal dynamics

ma_length = 366
ma_offset = Int(floor(ma_length/2))
moving_average(series,n) = [mean(@view series[i-n:i+n]) for i in n+1:length(series)-n]
dat_ma = DataFrame(datetime=dat.datetime[ma_offset+1:end-ma_offset], residual=dat.gauge[ma_offset+1:end-ma_offset] .- moving_average(dat.gauge, ma_offset))

In [ ]:
## group data by year and compute the annual maxima

dat_ma = dropmissing(dat_ma) # drop missing data
dat_annmax = combine(dat_ma -> dat_ma[argmax(dat_ma.residual), :], groupby(transform(dat_ma, :datetime => x->year.(x)), :datetime_function))
delete!(dat_annmax, nrow(dat_annmax)) # delete 2023; haven't seen much of that year yet
rename!(dat_annmax, :datetime_function => :Year)
select!(dat_annmax, [:Year, :residual])

## Problems (100 points)

### Problem 1: Stationary Model (40 points)

In this problem, you will fit a stationary GEV model and compute the Akaike and Deviance Information Criteria. You will also look at relevant graphical checks.

#### Problem 1.1: Fit the model (15 points)

Construct a `Turing.jl` stationary model with appropriate priors (there's no right choice; remember that thinking about the priors is part of the model checking process). Find the MLE estimate (you'll need this for the AIC) and use MCMC to sample from the posterior.

#### Problem 1.2: Compute information criteria (10 points)

Compute AIC and DIC for the stationary model.

### Problem 1.3: Graphical checks (15 points)

Conduct relevant graphical checks you can think of. These could include return periods, credible intervals for the data, or any other statistics that seem appropriate. Explain what each check is for and what your conclusions are about the model based on them. What would you change about the model in 1.1, if anything, based on these checks?

### Problem 2: Nonstationary Model (40 points)

Next, we will model the tidal extremes using a non-stationary GEV distribution, where the location parameter (but not the shape or scale) is represented by a linear regression $$\mu_t = \mu_0 + \mu_1 x_t,$$ where $x_t$ is the annual mean [Pacific Decadal Oscillation (PDO)](https://climatedataguide.ucar.edu/climate-data/pacific-decadal-oscillation-pdo-definition-and-indices) index, which is based on the variability of sea-surface temperatures (SSTs) in the North Pacific (versus the El Niño–Southern Oscillation (ENSO), which emphasizes the equatorial SSTs).

First, let's load the [PDO index dataset from NOAA](https://www.ncei.noaa.gov/access/monitoring/pdo/).

In [ ]:
## load the data from the file and return a DataFrame of DateTime values and gauge measurements

function load_pdo(fname)
    # This uses the DataFramesMeta.jl package, which makes it easy to string together commands to load and process data
    df = CSV.read(fname, DataFrame; delim=" ", header=2, ignorerepeated=true)
    # take yearly average
    @transform!(df, :PDO = mean(AsTable(names(df)[2:13])))
    @select!(df, $[:Year, :PDO])
    @rsubset!(df, :Year != 2023)
    return df
end

pdo = load_pdo("data/ersst.v5.pdo.dat")
# subset for years that match the tide gauge data
years = dat_annmax[!, :Year]
@rsubset!(pdo, :Year in years)

#### Problem 2.1: Fit the model (15 points)

Construct a `Turing.jl` nonstationary model with appropriate priors (there's no right choice; remember that thinking about the priors is part of the model checking process). Find the MLE estimate (you'll need this for the AIC) and use MCMC to sample from the posterior.

#### Problem 2.2: Compute information criteria (10 points)

Compute AIC and DIC for the nonstationary model.

### Problem 2.3: Graphical checks (15 points)

Conduct relevant graphical checks you can think of. These could include return periods, credible intervals for the data, or any other statistics that seem appropriate. Explain what each check is for and what your conclusions are about the model based on them. What would you change about the model in 1.1, if anything, based on these checks?

### Problem 3 (20 points)

Based on the information criteria and your graphical checks, what do you think is the relative evidence for dependence of the San Francisco tide gauge extremes on the PDO? Can you draw a conclusion about which model is better? What else could you try?